In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
import spams
import numpy as np
import skimage 
import tensorflow as tf
import math as m
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import layers
from mxnet import nd
import mxnet as mx
from tensorflow import keras
from sklearn.cluster import KMeans

2023-07-04 00:31:24.194722: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [3]:
data = np.load('/home/xipeng/FLcode/organamnist.npz')
data.files

x_train = data["train_images"]
y_train = data["train_labels"]
x_test = data["test_images"]
y_test = data["test_labels"]

# 为图像数据添加通道维度
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# 归一化图像数据
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0



import random

# 设备数量和IID程度
num_devices = 30
iid_ratio = 0.5

# 计算每个设备应包含的类别数量
num_classes = len(np.unique(y_train))
samples_per_device = len(x_train) // num_devices
selected_classes_per_device = int(num_classes * iid_ratio)

# 分配数据
device_data = [[] for _ in range(num_devices)]

# 为每个设备分配数据
for device_id in range(num_devices):
    # 随机选择类别
    random_classes = list(range(num_classes))
    random.shuffle(random_classes)
    selected_classes = random_classes[:selected_classes_per_device]

    # 获取已选类别的索引
    selected_indices = []
    for cls in selected_classes:
        class_indices = np.where(np.array(y_train) == cls)[0]
        selected_indices.extend(class_indices)

    # 在已选类别的索引中进行随机抽样
    random_selected_indices = random.sample(selected_indices, samples_per_device)
    device_data[device_id] = [(x_train[i], y_train[i]) for i in random_selected_indices]
y_train = keras.utils.to_categorical(y_train, num_classes)
# 检查每个设备上的样本数量
for i in range(num_devices):
    print(f"Device {i}: {len(device_data[i])} samples")
    print(f"Device {i}: ", np.unique(np.array([label for _, label in device_data[i]]), return_counts=True))  
data_x = [np.array([sample for sample, _ in device_data[i]]) for i in range(num_devices)]
data_y = [np.array([label for _, label in device_data[i]]) for i in range(num_devices)]
data_y = keras.utils.to_categorical(data_y, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = data_x
y_train = data_y

Device 0: 1152 samples
Device 0:  (array([ 1,  6,  7,  8, 10], dtype=uint8), array([ 93, 383, 220, 229, 227]))
Device 1: 1152 samples
Device 1:  (array([ 1,  5,  7,  9, 10], dtype=uint8), array([119, 292, 284, 217, 240]))
Device 2: 1152 samples
Device 2:  (array([2, 3, 6, 8, 9], dtype=uint8), array([ 83, 127, 434, 274, 234]))
Device 3: 1152 samples
Device 3:  (array([ 0,  3,  4,  6, 10], dtype=uint8), array([125, 107, 276, 409, 235]))
Device 4: 1152 samples
Device 4:  (array([1, 4, 6, 7, 9], dtype=uint8), array([ 85, 243, 389, 251, 184]))
Device 5: 1152 samples
Device 5:  (array([2, 3, 4, 5, 9], dtype=uint8), array([106, 116, 335, 324, 271]))
Device 6: 1152 samples
Device 6:  (array([1, 5, 6, 8, 9], dtype=uint8), array([ 85, 226, 406, 233, 202]))
Device 7: 1152 samples
Device 7:  (array([0, 1, 2, 3, 4], dtype=uint8), array([231, 157, 160, 161, 443]))
Device 8: 1152 samples
Device 8:  (array([ 3,  4,  7,  9, 10], dtype=uint8), array([110, 304, 261, 215, 262]))
Device 9: 1152 samples
Dev

In [4]:
n=30

In [5]:
def  set_model():
    model=Sequential()
        # 定义顺序模型           
    model.add(Convolution2D(
        input_shape = (28,28,1),
        filters = 8,
        kernel_size = 3,
        strides = 1,
        padding = 'same',
        activation = 'relu',
    ))
    # 第一个池化层
    model.add(MaxPooling2D(
        pool_size = 2,
        strides = 2,
        padding = 'same',
    ))
    # 第二个卷积层
    model.add(Convolution2D(16,3,strides=1,padding='same',activation='relu'))
    # 第二个池化层
    model.add(MaxPooling2D(2,2,'same'))                                    
                  
       # 第二个卷积层      
    # 把第三个池化层的输出扁平化为1维
    model.add(layers.Flatten())
    # 第一个全连接层
    model.add(Dense(256,activation='relu'))
    # Dropout
    model.add(Dropout(0.5)) 
    # 第二个全连接层
    model.add(Dense(num_classes,activation='softmax'))

    # 定义优化器
    adam = Adam(lr =5e-4)        
        
    # 定义优化器,loss function,训练过程中计算准确率
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

        #print(model_d[i])
    return model

In [6]:
epochs_parameters_gradients = []
num_epochs = 20
from keras import backend as K
loss_function = CategoricalCrossentropy()
def federated_learning(models,data_x, data_y,att_k):
    client_parameters = []
    client_gradients = []
    
        # 遍历每个模型
    for i in range(len(models)):
        client_models_parameters = []
        client_models_gradients = []
            # 使用客户端数据训练模型
        models[i].fit(data_x[i], data_y[i],batch_size=16,epochs=3,verbose=0)
            # 获取参数和梯度
        weights = models[i].get_weights()
        with tf.GradientTape() as tape:
            predictions = models[i](data_x[i], training=True)
            loss = loss_function(data_y[i], predictions)
        gradients = tape.gradient(loss, models[i].trainable_weights)

            # 保存客户端的参数和梯度
        client_models_parameters.append(weights)
        client_models_gradients.append(gradients)

        # 保存客户端的模型参数和梯度
        client_parameters.append(client_models_parameters)
        client_gradients.append(client_models_gradients)
    for i in range(att_k):
        for j in range(8):
            client_parameters[i][0][j] = skimage.util.random_noise(client_parameters[i][0][j], mode="gaussian", var=0.2, clip=True) 
    
    # 聚合全局参数和梯度
    aggregated_global_parameters = [np.mean([params[i] for params in client_parameters], axis=0) for i in range(len(client_parameters[0]))]
    aggregated_global_gradients = [np.mean([grads[i] for grads in client_gradients], axis=0) for i in range(len(client_gradients[0]))]

    return models,aggregated_global_parameters, aggregated_global_gradients,client_parameters,client_gradients

In [7]:
def fedavg(aggregated_global_parameters,model_list):
    for i in range (len(model_list)):
        model_list[i].set_weights(aggregated_global_parameters[0])
    loss,accuracy = model_list[0].evaluate(x_test,y_test,verbose=1)
    return model_list

In [8]:
def lbfgs(S_k_list, Y_k_list, v):
    curr_S_k = nd.concat(*[nd.expand_dims(x, axis=1) for x in S_k_list], dim=1)
    curr_Y_k = nd.concat(*[nd.expand_dims(x, axis=1) for x in Y_k_list], dim=1)

    S_k_time_Y_k = nd.dot(curr_S_k.T, curr_Y_k)
    S_k_time_S_k = nd.dot(curr_S_k.T, curr_S_k)
    R_k = np.triu(S_k_time_Y_k.asnumpy())
    
    L_k = S_k_time_Y_k - nd.array(R_k, ctx=mx.cpu())
    sigma_k = nd.dot(Y_k_list[-1].T, S_k_list[-1]) / (nd.dot(S_k_list[-1].T, S_k_list[-1]))
    D_k_diag = nd.diag(S_k_time_Y_k)
    upper_mat = nd.concat(*[sigma_k * S_k_time_S_k, L_k], dim=1)
    lower_mat = nd.concat(*[L_k.T, -nd.diag(D_k_diag)], dim=1)
    mat = nd.concat(*[upper_mat, lower_mat], dim=0)
    mat_inv = nd.linalg.inverse(mat)
    approx_prod = nd.array(sigma_k) * nd.array(v)
    #approx_prod = sigma_k * v
    #p_mat = nd.concat(*[nd.dot(curr_S_k.T, sigma_k * v), nd.dot(curr_Y_k.T, v)], dim=0)
    p_mat = nd.concat(*[nd.dot(curr_S_k.T, sigma_k * nd.array(v)), nd.dot(curr_Y_k.T, nd.array(v))], dim=0)

    approx_prod -= nd.dot(nd.dot(nd.concat(*[sigma_k * curr_S_k, curr_Y_k], dim=1), mat_inv), p_mat)

    return approx_prod


In [9]:
from mxnet import nd, autograd, gluon

In [10]:
# def lbfgs(S_k_list, Y_k_list, v):
#     gpu_device = 2  # 您可以根据需要修改这个值
#     curr_S_k = nd.concat(*S_k_list, dim=1)
#     curr_Y_k = nd.concat(*Y_k_list, dim=1)
#     S_k_time_Y_k = nd.dot(curr_S_k.T, curr_Y_k)
#     S_k_time_S_k = nd.dot(curr_S_k.T, curr_S_k)
#     R_k = np.triu(S_k_time_Y_k.asnumpy())
#     L_k = S_k_time_Y_k - nd.array(R_k, ctx=mx.gpu(gpu_device))
#     sigma_k = nd.dot(Y_k_list[-1].T, S_k_list[-1]) / (nd.dot(S_k_list[-1].T, S_k_list[-1]))
#     D_k_diag = nd.diag(S_k_time_Y_k)
#     upper_mat = nd.concat(*[sigma_k * S_k_time_S_k, L_k], dim=1)
#     lower_mat = nd.concat(*[L_k.T, -nd.diag(D_k_diag)], dim=1)
#     mat = nd.concat(*[upper_mat, lower_mat], dim=0)
#     mat_inv = nd.linalg.inverse(mat)

#     approx_prod = sigma_k * v
#     p_mat = nd.concat(*[nd.dot(curr_S_k.T, sigma_k * v), nd.dot(curr_Y_k.T, v)], dim=0)
#     approx_prod -= nd.dot(nd.dot(nd.concat(*[sigma_k * curr_S_k, curr_Y_k], dim=1), mat_inv), p_mat)

#     return approx_prod


In [11]:
########写一个函数，该函数可以把数据拉伸
def flattened_p(global_parameters):
    global_parameters_flattened = np.concatenate([param.flatten() for param in global_parameters[0]])
    return global_parameters_flattened

def flattened_g(global_gradients):
    global_gradients_numpy = [x.numpy() for x in global_gradients[0]]
    global_gradients_numpy = np.array(global_gradients_numpy)
    global_gradients_flattened = np.concatenate([param.flatten() for param in global_gradients_numpy])
    return global_gradients_flattened

In [12]:
def fedavg_2(client_parameters,client_gradients,v,w_queue,g_queue,grad):
    w_queue = [nd.array(x) for x in w_queue]
    g_queue = [nd.array(x) for x in g_queue]
    w_lsit= []
    g_lsit = []
    for i in range(10):
        w_lsit.append(w_queue[i])
        g_lsit.append(g_queue[i])
    
    
    pre_c_gradients = []
    pre_g = []
    distances = []
    for i in range(30):
        hess = lbfgs(w_lsit, g_lsit, v[i])
        hess_narry = hess.asnumpy()
        grad_np = np.array(grad[i])
        # 计算梯度之间的距离
        distance = np.linalg.norm(grad_np - hess_narry)
        distances.append(distance)
    
    
    distances = np.array(distances).reshape(-1, 1)
    # 假设我们想要将数据分为 3 个类别
    k = 2

    kmeans = KMeans(n_clusters=k, random_state=0).fit(distances)

    # kmeans.labels_ 将会告诉你每个数据点属于哪个类别
    cluster_centers = kmeans.cluster_centers_

    # 找到数值最小的聚类中心的索引
    min_cluster_index = np.argmin(cluster_centers)

    # 找到所有属于该类别的索引
    min_cluster_indices = np.where(kmeans.labels_ == min_cluster_index)[0]
    
    print(min_cluster_indices)
    
    filtered_client_parameters = [client_parameters[i] for i in min_cluster_indices]
    filtered_client_gradients = [client_gradients[i] for i in min_cluster_indices]
# 使用筛选后的客户端参数计算聚合的全局参数
    
    aggregated_global_parameters = [np.mean([params[i] for params in filtered_client_parameters], axis=0)for i in range(len(client_parameters[0]))]
    aggregated_global_gradients = [np.mean([params[i] for params in filtered_client_gradients], axis=0)for i in range(len(client_gradients[0]))]
    for i in range (30):
        model_list[i].set_weights(aggregated_global_parameters[0])
    
    return model_list,aggregated_global_parameters,aggregated_global_gradients

In [13]:
from collections import deque
w_queue = deque([], maxlen=10)  # 创建一个长度为10的双向队列
g_queue = deque([], maxlen=10)
print(type(w_queue))

<class 'collections.deque'>


In [14]:
import scipy
from scipy.io import loadmat

In [15]:
from collections import deque
w_queue = deque([], maxlen=10)  # 创建一个长度为10的双向队列
g_queue = deque([], maxlen=10)
global_parameters =[]
global_gradients=[]
client_parameters=[]
client_gradients =[]
loss_9 =[]
att_k = 9

model_list =[]
for i in range(30):
    model = set_model()
    model_list.append(model)
print(type(model_list))

model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
model_list = fedavg(global_parameters,model_list)

global_gradients_flattened =  flattened_g(global_gradients)
global_parameters_flattened =  flattened_p(global_parameters)
last_global_gradients_flattened = global_gradients_flattened
last_global_parameters_flattened = global_parameters_flattened



for i in range (10):
    model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
    model_list = fedavg(global_parameters,model_list)
    global_gradients_flattened =  flattened_g(global_gradients)
    global_parameters_flattened =  flattened_p(global_parameters)
    w_queue.append(global_parameters_flattened- last_global_parameters_flattened)
    g_queue.append(global_gradients_flattened- last_global_gradients_flattened)
    #########################这里的代码有问题，for i in range(len(global_parameters))，应该是一个，记得修改
    last_global_gradients_flattened = global_gradients_flattened
    last_global_parameters_flattened = global_parameters_flattened
    #############################v = para - last para
    last_para = []
    last_grad = []
    for i in range (30):
        last_grad.append(flattened_g(client_gradients[i])) 
        last_para.append(flattened_p(client_parameters[i])) 

        
for i in range (200):
    model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
    para = []
    grad = []
    print(i)
    for i in range (30):
        grad.append(flattened_g(client_gradients[i])) 
        para.append(flattened_p(client_parameters[i]))
    v = [current - last for current, last in zip(para, last_para)]
    model_list,global_parameters, global_gradients = fedavg_2(client_parameters,client_gradients,v,w_queue,g_queue,grad)
    global_gradients_flattened =  flattened_g(global_gradients)
    global_parameters_flattened =  flattened_p(global_parameters)
    w_queue.append(global_parameters_flattened- last_global_parameters_flattened)
    g_queue.append(global_gradients_flattened- last_global_gradients_flattened)
    #########################这里的代码有问题，for i in range(len(global_parameters))，应该是一个，记得修改
    last_global_gradients_flattened = global_gradients_flattened
    last_global_parameters_flattened = global_parameters_flattened
    #############################v = para - last para
    last_para = para
    last_grad = grad
    loss,accuracy = model_list[0].evaluate(x_test,y_test,verbose=1)
    loss_9.append(loss)
    scipy.io.savemat('loss_9.mat', mdict={'loss_9': loss_9,})

2023-07-04 00:31:30.176180: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-07-04 00:31:30.223875: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-07-04 00:31:30.223959: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-07-04 00:31:30.224069: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-07-04 00:31:30.224121: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-07-04 00:31:30.224178: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

<class 'list'>


2023-07-04 00:31:38.624034: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-07-04 00:31:39.050975: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2023-07-04 00:31:41.313802: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2023-07-04 00:31:41.543632: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


556/556 [==============================] - 2s 3ms/step - loss: 1.2069 - accuracy: 0.6264
0
[ 1  4  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.2625 - accuracy: 0.5854
1
[ 0  1  4  5  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.1407 - accuracy: 0.6314
2
[ 0  2  6  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.1944 - accuracy: 0.6611
3
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 1s 3ms/step - loss: 0.9366 - accuracy: 0.7035
4
[ 1  2  4  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.0949 - accuracy: 0.6706
5
[ 2  4  7  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [===

In [16]:
from collections import deque
w_queue = deque([], maxlen=10)  # 创建一个长度为10的双向队列
g_queue = deque([], maxlen=10)
global_parameters =[]
global_gradients=[]
client_parameters=[]
client_gradients =[]
loss_6 =[]
att_k = 6

model_list =[]
for i in range(30):
    model = set_model()
    model_list.append(model)
print(type(model_list))

model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
model_list = fedavg(global_parameters,model_list)

global_gradients_flattened =  flattened_g(global_gradients)
global_parameters_flattened =  flattened_p(global_parameters)
last_global_gradients_flattened = global_gradients_flattened
last_global_parameters_flattened = global_parameters_flattened



for i in range (10):
    model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
    model_list = fedavg(global_parameters,model_list)
    global_gradients_flattened =  flattened_g(global_gradients)
    global_parameters_flattened =  flattened_p(global_parameters)
    w_queue.append(global_parameters_flattened- last_global_parameters_flattened)
    g_queue.append(global_gradients_flattened- last_global_gradients_flattened)
    #########################这里的代码有问题，for i in range(len(global_parameters))，应该是一个，记得修改
    last_global_gradients_flattened = global_gradients_flattened
    last_global_parameters_flattened = global_parameters_flattened
    #############################v = para - last para
    last_para = []
    last_grad = []
    for i in range (30):
        last_grad.append(flattened_g(client_gradients[i])) 
        last_para.append(flattened_p(client_parameters[i])) 

        
for i in range (200):
    model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
    para = []
    grad = []
    print(i)
    for i in range (30):
        grad.append(flattened_g(client_gradients[i])) 
        para.append(flattened_p(client_parameters[i]))
    v = [current - last for current, last in zip(para, last_para)]
    model_list,global_parameters, global_gradients = fedavg_2(client_parameters,client_gradients,v,w_queue,g_queue,grad)
    global_gradients_flattened =  flattened_g(global_gradients)
    global_parameters_flattened =  flattened_p(global_parameters)
    w_queue.append(global_parameters_flattened- last_global_parameters_flattened)
    g_queue.append(global_gradients_flattened- last_global_gradients_flattened)
    #########################这里的代码有问题，for i in range(len(global_parameters))，应该是一个，记得修改
    last_global_gradients_flattened = global_gradients_flattened
    last_global_parameters_flattened = global_parameters_flattened
    #############################v = para - last para
    last_para = para
    last_grad = grad 
    loss,accuracy = model_list[0].evaluate(x_test,y_test,verbose=1)
    loss_6.append(loss)
    scipy.io.savemat('loss_6.mat', mdict={'loss_6': loss_6,})

<class 'list'>
556/556 [==============================] - 2s 3ms/step - loss: 1.3967 - accuracy: 0.5705
0
[ 0  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.3290 - accuracy: 0.5865
1
[ 2  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.2688 - accuracy: 0.5802
2
[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.0721 - accuracy: 0.6648
3
[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.0382 - accuracy: 0.6718
4
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.0367 - accuracy: 0.6581
5
[ 1  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22

In [17]:
#最后看一下c_gradients_flattened和pre_c_gradients的差值


In [18]:
from collections import deque
w_queue = deque([], maxlen=10)  # 创建一个长度为10的双向队列
g_queue = deque([], maxlen=10)
global_parameters =[]
global_gradients=[]
client_parameters=[]
client_gradients =[]
loss_3 =[]
att_k = 3

model_list =[]
for i in range(30):
    model = set_model()
    model_list.append(model)
print(type(model_list))

model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
model_list = fedavg(global_parameters,model_list)

global_gradients_flattened =  flattened_g(global_gradients)
global_parameters_flattened =  flattened_p(global_parameters)
last_global_gradients_flattened = global_gradients_flattened
last_global_parameters_flattened = global_parameters_flattened



for i in range (10):
    model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
    model_list = fedavg(global_parameters,model_list)
    global_gradients_flattened =  flattened_g(global_gradients)
    global_parameters_flattened =  flattened_p(global_parameters)
    w_queue.append(global_parameters_flattened- last_global_parameters_flattened)
    g_queue.append(global_gradients_flattened- last_global_gradients_flattened)
    #########################这里的代码有问题，for i in range(len(global_parameters))，应该是一个，记得修改
    last_global_gradients_flattened = global_gradients_flattened
    last_global_parameters_flattened = global_parameters_flattened
    #############################v = para - last para
    last_para = []
    last_grad = []
    for i in range (30):
        last_grad.append(flattened_g(client_gradients[i])) 
        last_para.append(flattened_p(client_parameters[i])) 

        
for i in range (200):
    model_list,global_parameters, global_gradients,client_parameters,client_gradients = federated_learning(model_list,x_train,y_train,att_k)
    para = []
    grad = []
    print(i)
    for i in range (30):
        grad.append(flattened_g(client_gradients[i])) 
        para.append(flattened_p(client_parameters[i]))
    v = [current - last for current, last in zip(para, last_para)]
    model_list,global_parameters, global_gradients = fedavg_2(client_parameters,client_gradients,v,w_queue,g_queue,grad)
    global_gradients_flattened =  flattened_g(global_gradients)
    global_parameters_flattened =  flattened_p(global_parameters)
    w_queue.append(global_parameters_flattened- last_global_parameters_flattened)
    g_queue.append(global_gradients_flattened- last_global_gradients_flattened)
    #########################这里的代码有问题，for i in range(len(global_parameters))，应该是一个，记得修改
    last_global_gradients_flattened = global_gradients_flattened
    last_global_parameters_flattened = global_parameters_flattened
    #############################v = para - last para
    last_para = para
    last_grad = grad 
    loss,accuracy = model_list[0].evaluate(x_test,y_test,verbose=1)
    loss_3.append(loss)
    scipy.io.savemat('loss_3.mat', mdict={'loss_3': loss_3,})


<class 'list'>
556/556 [==============================] - 2s 3ms/step - loss: 1.3521 - accuracy: 0.5631
0
[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.3787 - accuracy: 0.5666
1
[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.3226 - accuracy: 0.5723
2
[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.3853 - accuracy: 0.5563
3
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.2013 - accuracy: 0.6127
4
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29]
556/556 [==============================] - 1s 3ms/step - loss: 1.1605 - accuracy: 0.6261
5
[